# Netflix Titles — EDA & Cleaning Notebook

Summary: Cleaning and preprocessing the Netflix Titles dataset for analysis and modeling. This notebook includes steps to inspect data, handle missing values, engineer features, and save a finalized dataset ready for GitHub.

In [2]:
# Load core libraries and the dataset
import pandas as pd
import numpy as np
from datetime import datetime

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 200)

import os
print("Current working directory:", os.getcwd())

df = pd.read_csv("netflix_titles.csv")
print("Dataset loaded. Shape:", df.shape)
df.head()

Current working directory: c:\Users\spaa_\OneDrive\Desktop\New folder
Dataset loaded. Shape: (8807, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Ma...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Action & Adventure","To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war."
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV Comedies","In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life."


##  Initial inspection
- Checking columns, dtypes, missing values, and duplicates.

In [ ]:
# Initial inspection
df.info()
print('\nMissing values per column:')
print(df.isna().sum())
print('\nNumber of duplicates based on show_id:', df.duplicated(subset=['show_id']).sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB

Missing values per column:
show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in     

##  Removing exact duplicates (if any) and reset index

In [ ]:
# Remove exact duplicate rows (if any)
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print(f"Rows before: {before}, after dropping exact duplicates: {after}")

Rows before: 8807, after dropping exact duplicates: 8807


##  Handling missing values


In [ ]:
# Copy dataframe to work on
df_clean = df.copy()

# Fill strategies
# For textual descriptors where missing is acceptable, fill with explicit placeholder
for col in ['director', 'cast', 'country', 'rating', 'duration']:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna('Not Specified')

# For date_added, keep as NaT if missing but note count
print("Missing 'date_added' count:", df_clean['date_added'].isna().sum())

# Confirm
df_clean[['director','cast','country','rating','duration']].isna().sum()

Missing 'date_added' count: 10


director    0
cast        0
country     0
rating      0
duration    0
dtype: int64

## Normalizing text columns
- Trim whitespace and lowercase (where appropriate)

In [ ]:
# Strip whitespace and normalize case for textual cols
text_cols = ['type','title','director','cast','country','rating','listed_in','description']
for c in text_cols:
    if c in df_clean.columns:
        df_clean[c] = df_clean[c].astype(str).str.strip()

# Quick check
df_clean[['type','title','director']].head()

,type,title,director
0,Movie,Dick Johnson Is Dead,Kirsten Johnson
1,TV Show,Blood & Water,Not Specified
2,TV Show,Ganglands,Julien Leclercq
3,TV Show,Jailbirds New Orleans,Not Specified
4,TV Show,Kota Factory,Not Specified


##  Feature engineering
- Converting `duration` to numeric minutes or seasons
- Extracting `year_added`, `content_age`, and `is_movie`
- Extracting a primary genre from `listed_in`

In [ ]:
# Duration parsing
def parse_duration(d):
    d = str(d).strip()
    # If like '90 min' -> return 90
    if 'min' in d:
        try:
            return int(d.lower().replace('min','').strip())
        except:
            return np.nan
    # If like '2 Seasons' or '1 Season' -> return number of seasons (as negative to distinguish)
    if 'season' in d.lower():
        try:
            return int(d.split()[0])
        except:
            return np.nan
    return np.nan

if 'duration' in df_clean.columns:
    df_clean['duration_parsed'] = df_clean['duration'].apply(parse_duration)

# Distinguish movie minutes from seasons via a helper column
df_clean['is_movie'] = (df_clean['type'].str.lower() == 'movie').astype(int)

# year_added from date_added
if 'date_added' in df_clean.columns:
    df_clean['date_added'] = pd.to_datetime(df_clean['date_added'], errors='coerce')
    df_clean['year_added'] = df_clean['date_added'].dt.year

# content_age relative to 2025 (project year)
df_clean['content_age'] = 2025 - df_clean['release_year']

# primary genre (take first genre listed)
if 'listed_in' in df_clean.columns:
    df_clean['primary_genre'] = df_clean['listed_in'].apply(lambda x: str(x).split(',')[0].strip() if pd.notna(x) else 'Not Specified')

# Quick checks
df_clean[['duration','duration_parsed','is_movie','year_added','content_age','primary_genre']].head()

,duration,duration_parsed,is_movie,year_added,content_age,primary_genre
0,90 min,90.0,1,2021.0,5,Documentaries
1,2 Seasons,2.0,0,2021.0,4,International TV Shows
2,1 Season,1.0,0,2021.0,4,Crime TV Shows
3,1 Season,1.0,0,2021.0,4,Docuseries
4,2 Seasons,2.0,0,2021.0,4,International TV Shows


##  Data quality checks
- Checking for outliers, distribution of key fields, and missingness after transformations.

In [ ]:
# Check distributions and missingness
print("Duration parsed - count, mean, min, max:")
print(df_clean['duration_parsed'].describe())

print('\nValue counts: is_movie')
print(df_clean['is_movie'].value_counts())

print('\nTop 10 primary genres:')
print(df_clean['primary_genre'].value_counts().head(10))

# Missing after transformation
print('\nMissing after transforms:')
print(df_clean[['duration_parsed','year_added']].isna().sum())

Duration parsed - count, mean, min, max:
count    8804.000000
mean       69.846888
std        50.814828
min         1.000000
25%         2.000000
50%        88.000000
75%       106.000000
max       312.000000
Name: duration_parsed, dtype: float64

Value counts: is_movie
is_movie
1    6131
0    2676
Name: count, dtype: int64

Top 10 primary genres:
primary_genre
Dramas                      1600
Comedies                    1210
Action & Adventure           859
Documentaries                829
International TV Shows       774
Children & Family Movies     605
Crime TV Shows               399
Kids' TV                     388
Stand-Up Comedy              334
Horror Movies                275
Name: count, dtype: int64

Missing after transforms:
duration_parsed     3
year_added         98
dtype: int64


##  Save cleaned dataset
- Save as `netflix_titles_cleaned.csv` for GitHub upload and future modeling.

In [ ]:
import os
import pandas as pd

downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")

file_path = os.path.join(downloads_path, "netflix_titles_cleaned.csv")

df_clean.to_csv(file_path, index=False)

print(f"✅ Cleaned dataset saved to: {file_path}")

df_clean.shape


✅ Cleaned dataset saved to: C:\Users\spaa_\Downloads\netflix_titles_cleaned.csv


(8807, 17)